# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686566


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:  10%|▉         | 3/31 [00:06<01:03,  2.26s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:54,  2.02s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:40,  1.57s/it]

Rendering models:  19%|█▉        | 6/31 [00:09<00:31,  1.25s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:20<00:01,  1.51it/s]

Rendering models:  94%|█████████▎| 29/31 [00:20<00:01,  1.92it/s]

Rendering models:  97%|█████████▋| 30/31 [00:21<00:00,  2.14it/s]

Rendering models: 100%|██████████| 31/31 [00:21<00:00,  2.64it/s]

bimlesh                               0.000783
MissDonkey                            0.000192
not-logged-in-c5c35d38f95060274d07    0.008946
shirogane                             0.000163
SavinSpaceEngineer                    0.000105
djswanso                              0.000192
kjkavene                              0.000230
not-logged-in-ff40b895d06514783cbc    0.000729
justsomenoodles                       0.000180
jmfranci                              0.001984
ncaro                                 0.000782
not-logged-in-f7e2260ed4a9517a3e4a    0.033681
Galaxy-one                            0.000180
sn346808                              0.000321
sn345012                              0.003950
LeeJoseph                             0.011930
KrisKeogh                             0.003264
ElisabethB                            0.000153
Sylverone                             0.000282
puic                                  0.000171
H_Robinson                            0.000541
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())